# Using European Put and Calls as Regressors

In [9]:
# Black-Scholes formula for European call
bs_call <- function(S, K, r, T, sigma) {
  set.seed(123)
  d1 <- (log(S / K) + (r + 0.5 * sigma^2) * T) / (sigma * sqrt(T))
  d2 <- d1 - sigma * sqrt(T)
  S * pnorm(d1) - K * exp(-r * T) * pnorm(d2)
}

# Black-Scholes formula for European put
bs_put <- function(S, K, r, T, sigma) {
  set.seed(123)
  d1 <- (log(S / K) + (r + 0.5 * sigma^2) * T) / (sigma * sqrt(T))
  d2 <- d1 - sigma * sqrt(T)
  K * exp(-r * T) * pnorm(-d2) - S * pnorm(-d1)
}

price_american_put_longstaff_schwartz_MC_euro <- function(K, M, N, r, S0, sigma, polynomial) {
  dt <- 1 / M
  discount <- exp(-r * dt)  
  set.seed(123)
  Z <- matrix(rnorm(N * M), nrow = N, ncol = M)  
  S <- S0 * exp(sigma * sqrt(dt) * t(apply(Z, 1, cumsum)))
  
  Cash_flow <- matrix(0, nrow = N, ncol = M)
  Cash_flow[, M] <- pmax(K - S[, M], 0)
  
  for (m in M:2) {
    X <- S[, m - 1]
    T_remaining <- (M - m + 1) * dt

    call_bs <- bs_call(X, K, r, T_remaining, sigma)
    put_bs <- bs_put(X, K, r, T_remaining, sigma)

    df_reg <- data.frame(# Create regressors
      S = X,
      call = call_bs,
      put = put_bs,
      Y = Cash_flow[, m] * discount
    )

    df_reg[X > K, ] <- NA

    if (all(is.na(df_reg))) {
      Cash_flow[, m - 1] <- 0
      next
    }

    regression <- lm(polynomial, data = df_reg)

    immediate_exercise <- pmax(K - X, 0)

    df_pred <- data.frame(
      S = X,
      call = call_bs,
      put = put_bs
    )
    continuation <- predict(regression, newdata = df_pred)

    full_step <- cbind(continuation, immediate_exercise)
    full_step[immediate_exercise == 0, ] <- NA

    result_vector <- ifelse(
      is.na(full_step[, 2]), 0,
      ifelse(full_step[, 1] > full_step[, 2], 0, full_step[, 2])
    )
    Cash_flow[, m - 1] <- result_vector
  }
  
  # Discounting cash flows
  for (i in 1:nrow(Cash_flow)) {
    for (j in 1:ncol(Cash_flow)) {
      if (Cash_flow[i, j] != 0) {
        Cash_flow[i, j] <- Cash_flow[i, j] * round(exp(-r * j * dt), 5)
        if (j < ncol(Cash_flow)) {
          Cash_flow[i, (j + 1):ncol(Cash_flow)] <- 0
        }
        break
      }
    }
  }

  return(mean(rowSums(Cash_flow)))
}


### Very similar to the binomial tree results especially for very in-the-money calls 

In [ ]:
price_american_put_longstaff_schwartz_MC_euro( 
  K = 970,
  M = 50, 
  N = 1000,  
  r = 0.005290473,
  S0 = 931.8, 
  sigma = 0.01026789,
  polynomial = Y ~ S + call + put + I(S^2) + I(call^2) + I(put^2)
)
#38.2 Binomial Tree

[1] 38.15371

In [ ]:
price_american_put_longstaff_schwartz_MC_euro( #
  K = 940,
  M = 50, 
  N = 100000,  
  r = 0.005290473,
  S0 = 931.8, 
  sigma = 0.01026789,
  polynomial = Y ~ S + call + put + I(S^2) + I(call^2) + I(put^2)
)
# 8.2 Binomial Tree

[1] 8.196795

In [ ]:
price_american_put_longstaff_schwartz_MC_euro(
  K = 931.8,
  M = 50, 
  N = 1000000,  
  r = 0.005290473,
  S0 = 931.8, 
  sigma = 0.01026789,
  polynomial = Y ~ S + call + put + I(S^2) + I(call^2) + I(put^2)
)
# 2.3239 Binomial Tree

[1] 1.344847

In [12]:
price_american_put_longstaff_schwartz_MC_euro(
  K = 931.8,
  M = 50, 
  N = 1000000,  
  r = 0.005290473,
  S0 = 931.8, 
  sigma = 0.01026789,
  polynomial = Y ~ S + put + I(S^2) + I(put^2)
)

[1] 1.380086

In [13]:
price_american_put_longstaff_schwartz_MC_euro(
  K = 931.8,
  M = 50, 
  N = 1000000,  
  r = 0.005290473,
  S0 = 931.8, 
  sigma = 0.01026789,
  polynomial = Y ~ put + I(put^2)
)

[1] 1.2173